## Install and Import Libraries

In [ ]:
!pip install git+https://github.com/oracle/Skater.git

In [1]:
import pandas as pd
import heapq
import pickle
import numpy as np
import seaborn as sns
import random
import matplotlib.patches as mpatches
import matplotlib as mpl
import matplotlib.pyplot as plt
from datetime import time
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import cross_val_score
from sklearn import svm, datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
#from tabulate import tabulate
from sklearn import cluster
from sklearn.metrics import silhouette_score
from sklearn.linear_model import Perceptron
from timeit import timeit
from sklearn import datasets, tree
import datetime
import os
import warnings
warnings.simplefilter(action = 'ignore', category=FutureWarning)
warnings.filterwarnings('ignore')
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)
import pylab 
sns.set(style="ticks", color_codes=True, font_scale=1.5)
from matplotlib import pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from matplotlib.colors import ListedColormap
import matplotlib.colors as mcolors
%matplotlib inline
import mpl_toolkits
from mpl_toolkits.mplot3d import Axes3D
#from graphviz import Source
from IPython.display import Image
from scipy.stats import skew, norm, probplot, boxcox, f_oneway
from scipy import interp
from sklearn.base import BaseEstimator, TransformerMixin, clone, ClassifierMixin
from sklearn import metrics, tree
from sklearn.preprocessing import LabelEncoder, label_binarize, StandardScaler, PolynomialFeatures, MinMaxScaler
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold, cross_val_predict, train_test_split, RandomizedSearchCV
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics import roc_auc_score, roc_curve, auc, accuracy_score
from sklearn.linear_model import LogisticRegression
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.tree import DecisionTreeClassifier
#import xgboost as xgb
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.cluster import DBSCAN
#from mlxtend.classifier import StackingClassifier
from skater.core.local_interpretation.lime.lime_tabular import LimeTabularExplainer
from skater.core.explanations import Interpretation
from skater.model import InMemoryModel
import scipy
from sklearn.model_selection import GridSearchCV
from google.cloud import bigquery
from sklearn.model_selection import TimeSeriesSplit
from scipy.stats import chi2_contingency
import joblib

## Loading the Data

In [2]:
# set display options to show all columns
pd.set_option('display.max_columns', None)
# Set the float format to display numbers without scientific notation
pd.options.display.float_format = '{:.2f}'.format
# Set the client for future queries to BigQuery
client = bigquery.Client(project = "continente-lced-feup")
#data_table.enable_dataframe_formatter()
#auth.authenticate_user()

In [3]:
!gcloud auth application-default login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=w7lDzpz4CUAEUy7FqDWpSvKj7sSX5M&access_type=offline&code_challenge=NgAUzZkXJ6aChiu7terxj4D5e_YY6RD6Zr5QWK_0YqI&code_challenge_method=S256


Credentials saved to file: [/Users/henriqueribeiro/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "continente-lced-feup" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project

In [4]:
query = client.query("""
   SELECT *
   FROM 
       tables_staging.df_bold_model
    ORDER BY CUSTOMER_ACCOUNT_NR_MASK ASC, SUBCAT_CD_EXT ASC, YEAR ASC, MONTH ASC
   """)

df = query.result().to_dataframe() # Wait for the job to complete.
df

,CUSTOMER_ACCOUNT_NR_MASK,SUBCAT_CD_EXT,MONTH,YEAR,QUARTER,SEMESTER,CUST_NUM_TRANSACTIONS_MONTH,CUST_NUM_TRANSACTIONS_QUARTER,CUST_NUM_TRANSACTIONS_SEMESTER,CUST_NUM_TRANSACTIONS_YEAR,CUST_TOTAL_QTY_BOUGHT_MONTH,CUST_TOTAL_QTY_BOUGHT_QUARTER,CUST_TOTAL_QTY_BOUGHT_SEMESTER,CUST_TOTAL_QTY_BOUGHT_YEAR,CUST_NUM_UNIQUE_SUBCAT_MONTH,CUST_NUM_UNIQUE_SUBCAT_QUARTER,CUST_NUM_UNIQUE_SUBCAT_SEMESTER,CUST_NUM_UNIQUE_SUBCAT_YEAR,CUST_AVG_DAYS_SINCE_PRIOR_TRANSACTION_MONTH,CUST_AVG_DAYS_SINCE_PRIOR_TRANSACTION_QUARTER,CUST_AVG_DAYS_SINCE_PRIOR_TRANSACTION_SEMESTER,CUST_AVG_DAYS_SINCE_PRIOR_TRANSACTION_YEAR,REG_AVG_DAYS_SINCE_PRIOR_TRANSACTION_MONTH,CUST_AVG_BASKET_SIZE_MONTH,CUST_AVG_BASKET_SIZE_QUARTER,CUST_AVG_BASKET_SIZE_SEMESTER,CUST_AVG_BASKET_SIZE_YEAR,SUBCAT_NUM_TRANSACTIONS_MONTH,SUBCAT_NUM_TRANSACTIONS_QUARTER,SUBCAT_NUM_TRANSACTIONS_SEMESTER,SUBCAT_NUM_TRANSACTIONS_YEAR,SUBCAT_TOTAL_QTY_BOUGHT_MONTH,SUBCAT_TOTAL_QTY_BOUGHT_QUARTER,SUBCAT_TOTAL_QTY_BOUGHT_SEMESTER,SUBCAT_TOTAL_QTY_BOUGHT_YEAR,SUBCAT_NUM_UNIQUE_CUST_MONTH,SUBCAT_NUM_UNIQUE_CUST_QUARTER,SUBCAT_NUM_UNIQUE_CUST_SEMESTER,SUBCAT_NUM_UNIQUE_CUST_YEAR,CUSTSUBCAT_NUM_TRANSACTIONS_MONTH,CUSTSUBCAT_NUM_TRANSACTIONS_QUARTER,CUSTSUBCAT_NUM_TRANSACTIONS_SEMESTER,CUSTSUBCAT_NUM_TRANSACTIONS_YEAR,CUSTSUBCAT_TOTAL_QTY_BOUGHT_MONTH,CUSTSUBCAT_TOTAL_QTY_BOUGHT_QUARTER,CUSTSUBCAT_TOTAL_QTY_BOUGHT_SEMESTER,CUSTSUBCAT_TOTAL_QTY_BOUGHT_YEAR,CUSTSUBCAT_AVG_DAYS_SINCE_PRIOR_TRANSACTION_MONTH,CUSTSUBCAT_AVG_DAYS_SINCE_PRIOR_TRANSACTION_QUARTER,CUSTSUBCAT_AVG_DAYS_SINCE_PRIOR_TRANSACTION_SEMESTER,CUSTSUBCAT_AVG_DAYS_SINCE_PRIOR_TRANSACTION_YEAR,GENDER,FAMILY_MEMBERS,SEG_LIFESTYLE_CD,SEG_LIFESTAGE_CD,CAT_CD_EXT,TARGET
0,37912,10101,1,2021,1,1,5,5,5,5,24,24,24,24,12,12,12,12,2.25,2.25,2.25,2.25,4.20,3.60,3.60,3.60,3.60,2733,2733,2733,2733,4304,4304,4304,4304,1741,1741,1741,1741,0,0,0,0,0,0,0,0,0.00,0,0,0,M,"(3, 8)",2,4,101,0
1,37912,10101,2,2021,1,1,6,11,11,11,46,70,70,70,20,27,27,27,4.20,5.30,5.30,5.30,2.40,5.50,4.64,4.64,4.64,2640,5373,5373,5373,4140,8444,8444,8444,1676,2504,2504,2504,0,0,0,0,0,0,0,0,0.00,0,0,0,M,"(3, 8)",2,4,101,1
2,37912,10101,3,2021,1,1,11,22,22,22,60,130,130,130,21,37,37,37,2.40,3.86,3.86,3.86,5.20,3.55,4.09,4.09,4.09,2924,8297,8297,8297,4686,13130,13130,13130,1863,3090,3090,3090,2,2,2,2,3,3,3,3,9.00,9,9,9,M,"(3, 8)",2,4,101,1
3,37912,10101,4,2021,2,1,6,6,28,28,29,29,159,159,13,13,39,39,5.20,5.20,4.11,4.11,6.00,4.17,4.17,4.11,4.11,2577,2577,10874,10874,4200,4200,17330,17330,1760,1760,3511,3511,2,2,4,4,4,4,7,7,12.00,12,13,13,M,"(3, 8)",2,4,101,1
4,37912,10101,5,2021,2,1,5,11,33,33,31,60,190,190,13,20,44,44,6.00,5.50,4.38,4.38,3.57,4.20,4.18,4.12,4.12,2824,5401,13698,13698,4815,9015,22145,22145,1885,2731,3890,3890,1,3,5,5,1,5,8,8,0.00,20,17,17,M,"(3, 8)",2,4,101,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3365383,130366108,170313,8,2022,3,2,9,21,21,55,67,142,142,463,25,43,43,67,3.00,2.70,2.70,4.28,2.67,6.11,5.19,5.19,5.84,106,191,191,618,190,318,318,912,93,158,158,443,0,1,1,2,0,1,1,2,0.00,0,0,12,M,"(3, 8)",1,4,1703,0
3365384,130366108,170313,9,2022,3,2,10,31,31,65,76,218,218,539,25,51,51,69,2.67,2.93,2.93,4.14,4.00,5.80,5.39,5.39,5.83,67,258,258,685,89,407,407,1001,63,207,207,485,0,1,1,2,0,1,1,2,0.00,0,0,12,M,"(3, 8)",1,4,1703,0
3365385,130366108,170313,10,2022,4,2,7,7,38,72,18,18,236,557,10,10,54,70,4.00,4.00,3.27,4.20,4.80,1.86,1.86,4.74,5.44,78,78,336,763,118,118,525,1119,71,71,268,533,0,0,1,2,0,0,1,2,0.00,0,0,12,M,"(3, 8)",1,4,1703,0
3365386,130366108,170313,11,2022,4,2,6,13,44,78,54,72,290,611,18,24,57,71,4.80,4.33,3.47,4.23,2.75,4.33,3.00,4.68,5.36,77,155,413,840,105,223,630,1224,65,131,316,568,0,0,1,2,0,0,1,2,0.00,0,0,12,M,"(3, 8)",1,4,1703,0


In [5]:
query_products = client.query("""
   SELECT *
   FROM 
       tables_raw.dim_product
  LIMIT 6000000
   """)

pdct_df = query_products.result().to_dataframe() # Wait for the job to complete.

In [6]:
pdct_df = pdct_df[['SUBCAT_CD_EXT', 'SUBCAT_DSC_EXT']]

## Data Preparation

In [7]:


# Specify the columns to drop null values except for
columns_to_exclude = ['TARGET', 'REG_AVG_DAYS_SINCE_PRIOR_TRANSACTION_MONTH']

# Drop null values except for the specified columns
df = df.dropna(subset=[col for col in df.columns if col not in columns_to_exclude])


#df = df.dropna()
df = df.drop_duplicates()



In [8]:

df['fulldate'] = pd.to_datetime(df['MONTH'].astype(str) + '-' + df['YEAR'].astype(str))
# change from dtype datetime64[ns] to date time month
df['fulldate'] = df['fulldate'].dt.to_period('M')

In [9]:
df_sample = df.groupby('fulldate').sample(frac=1, random_state=101)

#drop customer id column because we want to generalize the model, instead of trying to predict for each customer
df_sample = df_sample.drop(columns=['CUSTOMER_ACCOUNT_NR_MASK','REG_AVG_DAYS_SINCE_PRIOR_TRANSACTION_MONTH'])

#df_500k.value_counts('fulldate').sort_index()

In [10]:
# Function to replace outliers with average value of the column
def replace_outliers_with_average(df_non_minmax, column):
    # Calculate the lower and upper bounds for outliers using the boxplot rule
    q1 = df_non_minmax[column].quantile(0.25)
    q3 = df_non_minmax[column].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    
    # Replace outliers with the average value of the column
    avg_value = df_non_minmax[column].mean()
    df_non_minmax[column] = df_non_minmax[column].apply(lambda x: avg_value if x < lower_bound or x > upper_bound else x)
    
    return df_non_minmax

# Detect outliers and replace them with average for each column in 'columns_to_check'
numerical_columns = ['CUST_NUM_TRANSACTIONS_MONTH', 'CUST_NUM_TRANSACTIONS_QUARTER',
       'CUST_NUM_TRANSACTIONS_SEMESTER', 'CUST_NUM_TRANSACTIONS_YEAR',
       'CUST_TOTAL_QTY_BOUGHT_MONTH', 'CUST_TOTAL_QTY_BOUGHT_QUARTER',
       'CUST_TOTAL_QTY_BOUGHT_SEMESTER', 'CUST_TOTAL_QTY_BOUGHT_YEAR',
       'CUST_NUM_UNIQUE_SUBCAT_MONTH', 'CUST_NUM_UNIQUE_SUBCAT_QUARTER',
       'CUST_NUM_UNIQUE_SUBCAT_SEMESTER', 'CUST_NUM_UNIQUE_SUBCAT_YEAR',
       'CUST_AVG_DAYS_SINCE_PRIOR_TRANSACTION_MONTH',
       'CUST_AVG_DAYS_SINCE_PRIOR_TRANSACTION_QUARTER',
       'CUST_AVG_DAYS_SINCE_PRIOR_TRANSACTION_SEMESTER',
       'CUST_AVG_DAYS_SINCE_PRIOR_TRANSACTION_YEAR',
       'CUST_AVG_BASKET_SIZE_MONTH', 'CUST_AVG_BASKET_SIZE_QUARTER',
       'CUST_AVG_BASKET_SIZE_SEMESTER', 'CUST_AVG_BASKET_SIZE_YEAR',
       'SUBCAT_NUM_TRANSACTIONS_MONTH', 'SUBCAT_NUM_TRANSACTIONS_QUARTER',
       'SUBCAT_NUM_TRANSACTIONS_SEMESTER', 'SUBCAT_NUM_TRANSACTIONS_YEAR',
       'SUBCAT_TOTAL_QTY_BOUGHT_MONTH', 'SUBCAT_TOTAL_QTY_BOUGHT_QUARTER',
       'SUBCAT_TOTAL_QTY_BOUGHT_SEMESTER', 'SUBCAT_TOTAL_QTY_BOUGHT_YEAR',
       'SUBCAT_NUM_UNIQUE_CUST_MONTH', 'SUBCAT_NUM_UNIQUE_CUST_QUARTER',
       'SUBCAT_NUM_UNIQUE_CUST_SEMESTER', 'SUBCAT_NUM_UNIQUE_CUST_YEAR',
       'CUSTSUBCAT_NUM_TRANSACTIONS_MONTH',
       'CUSTSUBCAT_NUM_TRANSACTIONS_QUARTER',
       'CUSTSUBCAT_NUM_TRANSACTIONS_SEMESTER',
       'CUSTSUBCAT_NUM_TRANSACTIONS_YEAR', 'CUSTSUBCAT_TOTAL_QTY_BOUGHT_MONTH',
       'CUSTSUBCAT_TOTAL_QTY_BOUGHT_QUARTER',
       'CUSTSUBCAT_TOTAL_QTY_BOUGHT_SEMESTER',
       'CUSTSUBCAT_TOTAL_QTY_BOUGHT_YEAR',
       'CUSTSUBCAT_AVG_DAYS_SINCE_PRIOR_TRANSACTION_MONTH',
       'CUSTSUBCAT_AVG_DAYS_SINCE_PRIOR_TRANSACTION_QUARTER',
       'CUSTSUBCAT_AVG_DAYS_SINCE_PRIOR_TRANSACTION_SEMESTER',
       'CUSTSUBCAT_AVG_DAYS_SINCE_PRIOR_TRANSACTION_YEAR']

for column in numerical_columns:
    df_sample = replace_outliers_with_average(df_sample, column)
    
df_sample.shape

(3365388, 56)

In [11]:
scaler = MinMaxScaler()
df_sample[numerical_columns] = scaler.fit_transform(df_sample[numerical_columns])

## Feature selection: Filter methods - No need to run

In [ ]:
# def our_heatmap(df_values, pv, threshold=1.1):
    
#     # Define the colors
#     colors = ['#84161a', '#fcf2f2']

#     # Create a list of relative positions for each color
#     positions = [0, 1]

#     # Create the custom colormap
#     cmap = mcolors.LinearSegmentedColormap.from_list("", list(zip(positions, colors)))

#     if pv:
#         # keep only the pvalues below the threshold
#         df_values = df_values[df_values < threshold]

#     else:
#         # keep only the correlations above the threshold
#         df_values = df_values[df_values >= threshold]

#     # Plot the heatmap with the custom colormap
#     fig, ax = plt.subplots(figsize=(20, 15))
#     sns.heatmap(df_values, annot=True, cmap=cmap, vmin=0, vmax=1)

In [ ]:
# # calculate correlation matrix
# corr = df[numerical_columns].corr()
# our_heatmap(corr, False, 0.8)

In [ ]:
# # Bootstrap sampling to identify associations/correlations between categorical variables

# categorical_columns = ['SUBCAT_CD_EXT','MONTH','QUARTER','SEMESTER','YEAR','GENDER','SEG_LIFESTYLE_CD','SEG_LIFESTAGE_CD','CAT_CD_EXT', 'FAMILY_MEMBERS']

# df_200k = df[categorical_columns]

# # Create an empty matrix to store the p-values
# pvals = np.zeros((len(df_200k.columns), len(df_200k.columns)))

# n_bootstraps = 1000

# pvals_ind = [None]*1000

# # Loop through all pairs of variables and calculate the p-value
# for i, var1 in enumerate(df_200k.columns):
#     for j, var2 in enumerate(df_200k.columns):
#         if i == j:
#             continue
#         else:
#             for k in range(0, n_bootstraps):
#                 sample = df_200k.sample(200, replace=True)
#                 cont_table = pd.crosstab(sample[var1], sample[var2])
#                 chi2, pval, dof, expected = chi2_contingency(cont_table)
#                 pvals_ind[k] = pval
        
#             pvals[i, j] = np.mean(pvals_ind)

# # Convert the matrix to a data frame and print the results
# pvals_df = pd.DataFrame(pvals, columns=df_200k.columns, index=df_200k.columns)

# pvals_df

In [ ]:
# our_heatmap(pvals_df)
# our_heatmap(pvals_df, 0.05)

## One-hot Encoding

In [12]:
df_sample = df_sample.drop(columns=['QUARTER','SEMESTER','SEG_LIFESTAGE_CD','SEG_LIFESTYLE_CD','MONTH', 'SUBCAT_CD_EXT'])

In [13]:
# One-hot encoding categorical variables
df_sample = pd.get_dummies(df_sample, columns=['GENDER','FAMILY_MEMBERS','CAT_CD_EXT','YEAR'])

In [34]:
df_sample['TARGET'].value_counts()

0    2755626
1     468762
Name: TARGET, dtype: Int64

## Outlier detection (unsupervised learning) - No need to run

In [ ]:
def plot_3d_clusters(df_clst, z_rot):
    # Compute the first three principal components of the data
    pca = PCA(n_components=3)
    principal_components = pca.fit_transform(df_clst)

    # Create a scatter plot of the first three principal components, colored by cluster
    cluster_colors = np.array(['#de1c26', 'yellow', 'orange', 'red', 'purple', 'black', 'gray', 'magenta', '#84161a'])
    sns.set_style("ticks", {"axes.facecolor": "#ffffff"})
    fig = plt.figure(figsize=(20, 15))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(principal_components[:,0], principal_components[:,1], principal_components[:,2], c=cluster_colors[df_clst['cluster'].values % len(cluster_colors)], alpha=0.8)
    ax.scatter(principal_components[outliers_mask, 0], principal_components[outliers_mask, 1], principal_components[outliers_mask, 2], c='#84161a', marker='X', s=100, alpha=1.0)
    ax.view_init(elev=10, azim=z_rot)
    
    ax.set_xlabel('PC 1')
    ax.set_ylabel('PC 2')
    ax.set_zlabel('PC 3')

    ax.w_xaxis.set_pane_color('#fcf2f2')  # Change x-axis color
    ax.w_yaxis.set_pane_color('#fcf2f2')  # Change y-axis color
    ax.w_zaxis.set_pane_color('#fcf2f2')  # Change z-axis color

    
    ax.w_xaxis.set_ticklabels([])
    ax.w_yaxis.set_ticklabels([])
    ax.w_zaxis.set_ticklabels([])
    #ax.grid(False)

    plt.show()


def plot_2d_clusters(df_clst):
    # Compute the first two principal components of the data
    pca = PCA(n_components=2)
    principal_components = pca.fit_transform(df_clst)

    # Create a scatter plot of the first three principal components, colored by cluster
    cluster_colors = np.array(['#de1c26', 'yellow', 'orange', 'red', 'purple', 'black', 'gray', 'magenta', '#84161a'])
    sns.set_style("ticks", {"axes.facecolor": "#ffffff"})
    fig = plt.figure(figsize=(15, 10))
    ax = fig.add_subplot(111)
    ax.scatter(principal_components[:,0], principal_components[:,1], c=cluster_colors[df_clst['cluster'].values % len(cluster_colors)], alpha=0.8)
    ax.scatter(principal_components[outliers_mask, 0], principal_components[outliers_mask, 1], c='#84161a', marker='X', s=100, alpha=1.0)

    ax.set_facecolor('#fcf2f2')
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    
    plt.xlabel('PC 1')
    plt.ylabel('PC 2')
    plt.show()

In [ ]:
# Extract the numerical columns
df_200k_out = df_200k[numerical_columns]

batch_size = 50000

while len(df_200k_out) > batch_size:
    df_50k = df_200k_out.sample(batch_size, replace=False)
    # Perform DBSCAN clustering and obtain cluster labels
    dbscan = DBSCAN(eps=0.6, min_samples=800)
    labels = dbscan.fit_predict(df_50k)

    # Identify the outliers
    outliers_mask = labels == -1
    outliers = df_50k[outliers_mask]
    df_200k_out = df_200k_out.drop(index=df_50k.index)
    df_200k = df_200k.drop(index=outliers.index)

    print("Number of outliers:", len(outliers))

    # Add cluster labels to the original dataframe
    df_50k['cluster'] = labels


# Perform DBSCAN clustering and obtain cluster labels
dbscan = DBSCAN(eps=0.6, min_samples=800)
labels = dbscan.fit_predict(df_200k_out)

# Identify the outliers
outliers_mask = labels == -1
outliers = df_200k_out[outliers_mask]
df_200k = df_200k.drop(index=outliers.index)

print("Number of outliers:", len(outliers))

# Add cluster labels to the original dataframe
df_200k_out['cluster'] = labels

plot_2d_clusters(df_200k_out)

plot_3d_clusters(df_200k_out, 0)
plot_3d_clusters(df_200k_out, 90)
plot_3d_clusters(df_200k_out, 180)
plot_3d_clusters(df_200k_out, 270)

## Machine Learning

In [ ]:
def get_results(model, gs ,name, data, true_labels, target_names = ['No buy', 'Buy'], results=None, reasume=False):
    global param
    
    if hasattr(model, 'layers'):
        param = wtp_dnn_model.history.params
        best = np.mean(history.history['val_accuracy'])
        # predicted_labels_test = np.argmax(model.predict(data) , axis=-1)
        # print(f'This is Predicted_Labels_test in the first if :{predicted_labels_test}')
        predicted_labels = (model.predict_proba(data)[:,1] >= 0.5).astype(int)
        print(f'\n\nThis is Predicted_Labels in the first if:{predicted_labels}\n\n')
        im_model = InMemoryModel(model.predict, examples=data, target_names=target_names)

    else:
        
        param = gs.best_params_
        best = gs.best_score_
        # predicted_labels_test = model.predict(data).ravel()
        # print(f'This is Predicted_Labels_test - else:{predicted_labels_test}')
        predicted_labels = (model.predict_proba(data)[:,1] >= 0.5).astype(int)
        print(f'\n\nThis is Predicted_Labels - else :{predicted_labels}\n\n')

        if hasattr(model, 'predict_proba'):
            #predicted_probs = model.predict_proba(data)[:, 1]
            #print(f'This is the predict_proba results but the ones from the get_results function: {predicted_probs}')
            print('This model has a predict_proba method')
        elif hasattr(model, 'decision_function'):
            im_model = InMemoryModel(model.decision_function, examples=data, target_names=target_names)
        else: 
            print('Cannot use InMemoryModel as predict_proba is not available')
           
        
    print('Mean Best Accuracy: {:2.2%}'.format(best))
    print('-'*60)
    print('Best Parameters:')
    print(param)
    print('-'*60)
    
    #y_pred = model.predict(data).ravel()

    #y_predict_class = [1 if prob > 0.4 else 0 for prob in y_predict_prob_class_1]

    # y_pred = (model.predict_proba(data)[:,1] >= 0.7).astype(bool)

    y_pred = (model.predict_proba(data)[:,1] >= 0.5).astype(int).ravel()
    #y_pred = int(y_pred)

    display_model_performance_metrics(true_labels, predicted_labels = predicted_labels, target_names = target_names)
    if len(target_names)==2:
        ras = roc_auc_score(y_true=true_labels, y_score=y_pred)
    else:
        roc_auc_multiclass, ras = roc_auc_score_multiclass(y_true=true_labels, y_score=y_pred, target_names=target_names)
        print('\nROC AUC Score by Classes:\n',roc_auc_multiclass)
        print('-'*60)

    print('\n\n              ROC AUC Score: {:2.2%}'.format(ras))
    prob, score_roc, roc_auc = plot_model_roc_curve(model, data, true_labels, label_encoder=None, class_names=target_names)
    
    #interpreter = Interpretation(data, feature_names=cols)
    #plots = interpreter.feature_importance.plot_feature_importance(im_model, progressbar=False, n_jobs=1, ascending=True)
    
    r1 = pd.DataFrame([(prob, best, np.round(accuracy_score(true_labels, predicted_labels), 4), 
                         ras, roc_auc)], index = [name],
                         columns = ['Prob', 'CV Accuracy', 'Accuracy', 'ROC AUC Score', 'ROC Area'])
    if reasume:
        results = r1
    elif (name in results.index):        
        results.loc[[name], :] = r1
    else: 
        results = results.append(r1)
        
    return y_pred, results 


def roc_auc_score_multiclass(y_true, y_score, target_names, average = "macro"):

  #creating a set of all the unique classes using the actual class list
  unique_class = set(y_true)
  roc_auc_dict = {}
  mean_roc_auc = 0
  for per_class in unique_class:
    #creating a list of all the classes except the current class 
    other_class = [x for x in unique_class if x != per_class]

    #marking the current class as 1 and all other classes as 0
    new_y_true = [0 if x in other_class else 1 for x in y_true]
    new_y_score = [0 if x in other_class else 1 for x in y_score]
    num_new_y_true = sum(new_y_true)

    #using the sklearn metrics method to calculate the roc_auc_score
    roc_auc = roc_auc_score(new_y_true, new_y_score, average = average)
    roc_auc_dict[target_names[per_class]] = np.round(roc_auc, 4)
    mean_roc_auc += num_new_y_true * np.round(roc_auc, 4)
    
  mean_roc_auc = mean_roc_auc/len(y_true)  
  return roc_auc_dict, mean_roc_auc

def get_metrics(true_labels, predicted_labels):
    global accuracy
    global precision
    global recall
    global f1
    accuracy = metrics.accuracy_score(true_labels, predicted_labels)
    precision = metrics.precision_score(true_labels, predicted_labels)
    recall = metrics.recall_score(true_labels, predicted_labels)
    f1 = metrics.f1_score(true_labels, predicted_labels)
    
    print('Accuracy:  {:2.2%} '.format(accuracy))
    print('Precision: {:2.2%} '.format(precision))
    print('Recall:    {:2.2%} '.format(recall))
    print('F1 Score:  {:2.2%} '.format(f1))
    # #append results to arrays
    # np.append(accuracy_array, metrics.accuracy_score(true_labels, predicted_labels))
    # np.append(precision_array, metrics.precision_score(true_labels, predicted_labels, average='weighted'))
    # np.append(recall_array, metrics.recall_score(true_labels, predicted_labels, average='weighted'))
    # np.append(f1_array, metrics.f1_score(true_labels, predicted_labels, average='weighted'))
    
                        

def train_predict_model(classifier,  train_features, train_labels,  test_features, test_labels):
    # build model    
    classifier.fit(train_features, train_labels)
    # predict using model
    predictions = classifier.predict(test_features) 
    return predictions    


def display_confusion_matrix(true_labels, predicted_labels, target_names):
    
    total_classes = len(target_names)
    level_labels = [total_classes*[0], list(range(total_classes))]

    cm = metrics.confusion_matrix(y_true=true_labels, y_pred=predicted_labels)
    cm_frame = pd.DataFrame(data=cm, 
                            columns=pd.MultiIndex(levels=[['Predicted:'], target_names], codes=level_labels), 
                            index=pd.MultiIndex(levels=[['Actual:'], target_names], codes=level_labels)) 
    print(cm_frame) 
    
def display_classification_report(true_labels, predicted_labels, target_names):

    report = metrics.classification_report(y_true=true_labels, y_pred=predicted_labels, target_names=target_names) 
    print(report)
    
def display_model_performance_metrics(true_labels, predicted_labels, target_names):
    print('Model Performance metrics:')
    print('-'*30)
    get_metrics(true_labels=true_labels, predicted_labels=predicted_labels)
    print('\nModel Classification report:')
    print('-'*30)
    display_classification_report(true_labels=true_labels, predicted_labels=predicted_labels, target_names=target_names)
    print('\nPrediction Confusion Matrix:')
    print('-'*30)
    display_confusion_matrix(true_labels=true_labels, predicted_labels=predicted_labels, target_names=target_names)

def plot_model_roc_curve(clf, features, true_labels, label_encoder=None, class_names=None):
    
    ## Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    if hasattr(clf, 'classes_'):
        class_labels = clf.classes_
    elif label_encoder:
        class_labels = label_encoder.classes_
    elif class_names:
        class_labels = class_names
    else:
        raise ValueError('Unable to derive prediction classes, please specify class_names!')
    n_classes = len(class_labels)
   
    if n_classes == 2:
        if hasattr(clf, 'predict_proba'):
            prb = clf.predict_proba(features)
            if prb.shape[1] > 1:
                y_score = prb[:, prb.shape[1]-1] 
            else:
                y_score = clf.predict(features).ravel()
            prob = True
        elif hasattr(clf, 'decision_function'):
            y_score = clf.decision_function(features)
            prob = False
        else:
            print("\n")
            #raise AttributeError("Estimator doesn't have a probability or confidence scoring system!")
        
        fpr, tpr, _ = roc_curve(true_labels, y_score)      
        roc_auc = auc(fpr, tpr)

        plt.plot(fpr, tpr, label='ROC curve (area = {0:3.2%})'.format(roc_auc), linewidth=2.5)
        
    elif n_classes > 2:
        if  hasattr(clf, 'clfs_'):
            y_labels = label_binarize(true_labels, classes=list(range(len(class_labels))))
        else:
            y_labels = label_binarize(true_labels, classes=class_labels)
        if hasattr(clf, 'predict_proba'):
            y_score = clf.predict_proba(features)
            prob = True
        elif hasattr(clf, 'decision_function'):
            y_score = clf.decision_function(features)
            prob = False
        else:
            raise AttributeError("Estimator doesn't have a probability or confidence scoring system!")
            
        for i in range(n_classes):
            fpr[i], tpr[i], _ = roc_curve(y_labels[:, i], y_score[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i])

        ## Compute micro-average ROC curve and ROC area
        fpr["micro"], tpr["micro"], _ = roc_curve(y_labels.ravel(), y_score.ravel())
        roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

        ## Compute macro-average ROC curve and ROC area
        # First aggregate all false positive rates
        all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))
        # Then interpolate all ROC curves at this points
        mean_tpr = np.zeros_like(all_fpr)
        for i in range(n_classes):
            mean_tpr += interp(all_fpr, fpr[i], tpr[i])
        # Finally average it and compute AUC
        mean_tpr /= n_classes
        fpr["macro"] = all_fpr
        tpr["macro"] = mean_tpr
        roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

        ## Plot ROC curves
        plt.figure(figsize=(12, 6))
        plt.plot(fpr["micro"], tpr["micro"], label='micro-average ROC curve (area = {0:2.2%})'
                       ''.format(roc_auc["micro"]), linewidth=3)

        plt.plot(fpr["macro"], tpr["macro"], label='macro-average ROC curve (area = {0:2.2%})'
                       ''.format(roc_auc["macro"]), linewidth=3)
  
        for i, label in enumerate(class_names):
            plt.plot(fpr[i], tpr[i], label='ROC curve of class {0} (area = {1:2.2%})'
                                           ''.format(label, roc_auc[i]), linewidth=2, linestyle=':')
            
        roc_auc = roc_auc["macro"]   
    else:
        raise ValueError('Number of classes should be atleast 2 or more')
        
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([-0.01, 1.0])
    plt.ylim([0.0, 1.01])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc="lower right")

    
    return prob, y_score, roc_auc

In [ ]:
class select_fetaures(object):  # BaseEstimator, TransformerMixin,
     def __init__(self, select_cols):
         self.select_cols_ = select_cols

     def fit(self, X, Y):
         pass

     def transform(self, X):
         return X.loc[:, self.select_cols_]

     def fit_transform(self, X, Y):
         self.fit(X, Y)
         df = self.transform(X)
         return df

     def __getitem__(self, x):
         return self.X[x], self.Y[x]

In [ ]:
def LR(X_train, y_train, X_train_tune, y_train_tune, X_test, y_test):
        
    clf = Pipeline([
                # ('pca', PCA()),
                ('clf', LogisticRegression())])  

    param_grid = {}

    # Use SequentialFeatureSelector for forward/backward selection
    sfs_backward = SequentialFeatureSelector(clf, n_features_to_select='auto', tol=0.00, direction='forward', scoring='precision', cv=None, n_jobs=-1)  # Does 5-Fold CV

    # Fit the feature selector to the training data
    sfs_backward.fit(X_train_tune, y_train_tune)

    # Get the selected features and transform the data
    X_train = sfs_backward.transform(X_train)
    X_test = sfs_backward.transform(X_test)

    global features_selected
    features_selected = sfs_backward.feature_names_in_[sfs_backward.support_]

    print('Number of selected features: {}'.format(sfs_backward.n_features_to_select_))
    print('Features selected: {}'.format(sfs_backward.feature_names_in_[sfs_backward.support_]))
        
    gs = GridSearchCV(estimator=clf, param_grid=param_grid, scoring='precision', cv=2, verbose=1, n_jobs=-1)
    LR = Pipeline([('gs', gs)]) 
        
    LR.fit(X_train,y_train)
    
    #print the number of features used in the model
    #print('Number of features used: {}'.format(LR.named_steps['gs'].best_estimator_.named_steps['clf'].coef_.shape[1]))

    joblib.dump(LR, 'LR.joblib')


    global y_proba
    y_proba = LR.predict_proba(X_test)

    

    y_pred, results = get_results(LR, gs ,'Logistic Regression', X_test, y_test, reasume=True)
    print(results)
    print(f'This is the predict_proba results {y_proba}')

    return y_pred
        

def RF(X_train, y_train, X_train_tune, y_train_tune, X_test, y_test):

    clf = Pipeline([
                # ('pca', PCA()),
                ('clf', RandomForestClassifier())])  

    param_grid = {'clf__criterion': ['gini']  # , 'entropy', 'log_loss'
                  ,'clf__n_estimators':  [500]       
                  ,'clf__min_samples_split': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
                  ,'clf__max_depth': [10, 50, 100, 150, 200]
                  ,'clf__min_samples_leaf': [20, 40, 60, 80, 100]}
    
    # Use SequentialFeatureSelector for forward/backward selection
    sfs_backward = SequentialFeatureSelector(clf, n_features_to_select='auto', tol=0.00, direction='forward', scoring='recall', cv=None, n_jobs=-1)  # Does 5-Fold CV

    # Fit the feature selector to the training data
    sfs_backward.fit(X_train_tune, y_train_tune)

    global features_selected
    features_selected = sfs_backward.feature_names_in_[sfs_backward.support_]

    # Get the selected features and transform the data
    X_train = sfs_backward.transform(X_train)
    X_test = sfs_backward.transform(X_test)

    print('Number of selected features: {}'.format(sfs_backward.n_features_to_select_))
    print('Features selected: {}'.format(sfs_backward.feature_names_in_[sfs_backward.support_]))

    gs = GridSearchCV(estimator=clf, param_grid=param_grid, scoring='recall', cv=2, verbose=1, n_jobs=-1)
    RF = Pipeline([('gs', gs)]) 
        
    RF.fit(X_train,y_train)

    joblib.dump(RF, 'RF.joblib')


    global y_proba
    y_proba = RF.predict_proba(X_test)

    y_pred, results = get_results(RF, gs ,'Random Forest', X_test, y_test, reasume=True)
    print(results)
    print(f'This is the predict_proba results {y_proba}')

    return y_pred


def SVM(X_train, y_train, X_train_tuning, y_train_tuning, X_test, y_test):

    clf = Pipeline([
                # ('pca', PCA()),
                ('clf', svm.SVC())])  

    param_grid = {'clf__C': [0.05, 0.1, 0.15, 0.2]  # 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0
                  ,'clf__kernel' : ['rbf']}   #, 'linear', , 'poly', 'linear', 'sigmoid', 'precomputed'

    # Use SequentialFeatureSelector for backward selection
    sfs_backward = SequentialFeatureSelector(clf, n_features_to_select='auto', tol=0.05, direction='forward', scoring='f1', cv=None, n_jobs=-1)  # Does 5-Fold CV

    # Fit the feature selector to the training data
    sfs_backward.fit(X_train_tuning, y_train_tuning)

    # Get the selected features and transform the data
    X_train = sfs_backward.transform(X_train)
    X_test = sfs_backward.transform(X_test)

    print('Number of selected features: {}'.format(sfs_backward.n_features_to_select_))
    print('Features selected: {}'.format(sfs_backward.feature_names_in_[sfs_backward.support_]))
        
    gs = GridSearchCV(estimator=clf, param_grid=param_grid, scoring='f1', cv=5, verbose=1, n_jobs=-1)
    SVM = Pipeline([('gs', gs)]) 
        
    SVM.fit(X_train,y_train)

    results = get_results(SVM, gs ,'Support Vector Machines', X_test, y_test, reasume=True)
    print(results)
    

def ANN(X_train, y_train, X_train_tune, y_train_tune, X_test, y_test):
    
    clf = Pipeline([
            # ('pca', PCA()),
            ('clf', MLPClassifier())])  

    param_grid = {'clf__hidden_layer_sizes': [(64,), (128,), (256,)],
                  'clf__activation': ['relu'],
                  'clf__solver': ['adam'],
                  'clf__early_stopping': [True],  # creates a stratified validation set (10% of training data)
                  'clf__validation_fraction': [0.2], 
                  'clf__n_iter_no_change': [10],
                  'clf__alpha': [0.0001, 0.001],
                  'clf__learning_rate': ['constant','invscaling','adaptive'],
                  'clf__tol': [0.0001, 0.001],
                  'clf__learning_rate_init': [0.0001, 0.001],
                  'clf__max_iter': [1000]}

    # Use SequentialFeatureSelector for forward/backward selection
    sfs_backward = SequentialFeatureSelector(clf, n_features_to_select='auto', tol=0.00, direction='forward', scoring='f1', cv=None, n_jobs=-1)  # Does 5-Fold CV

    # Fit the feature selector to the training data
    sfs_backward.fit(X_train_tune, y_train_tune)

    # Get the selected features and transform the data
    X_train = sfs_backward.transform(X_train)
    X_test = sfs_backward.transform(X_test)

    print('Number of selected features: {}'.format(sfs_backward.n_features_to_select_))
    print('Features selected: {}'.format(sfs_backward.feature_names_in_[sfs_backward.support_]))

    gs = GridSearchCV(estimator=clf, param_grid=param_grid, scoring='f1', cv=2, verbose=1, n_jobs=-1)
    ANN = Pipeline([('gs', gs)]) 
        
    ANN.fit(X_train,y_train)

    results = get_results(ANN, gs ,'Neural Network', X_test, y_test, reasume=True)
    print(results)

In [ ]:
def cv_12months(df_ML, model, resampling_tecnique):
    timestamps = df_ML['fulldate'].sort_values().unique()
    #display(timestamps.size)

    # find the minimum timestamp
    min_timestamp = timestamps[0]

    #store each of the metrics in an array for each one
    accuracy_array = np.array([])
    precision_array = np.array([])
    recall_array = np.array([])
    f1_array = np.array([])

    
    probability_array = np.array([])

    # full_xtest_df = []

    df_total = pd.DataFrame()


    full_test_count = 0
    

    
    # iterate over the timestamps and check if the time difference is less than 12 months
    for ts in timestamps[:-3]:
        
        if ts + 3 < timestamps.max():
            print(f'\n\nStart Iteration {ts}\n\n')
            if (ts - min_timestamp).n < 2: # 11
               
                print(f'Treino feito com os meses: {ts} até {ts+2}') # 11   
                train = df_ML[df_ML['fulldate'].isin([ts, ts+1, ts+2])]  # ts+3, ts+4, ts+5, ts+6, ts+7, ts+8, ts+9, ts+10, ts+11
                train_tune = train.groupby('fulldate').sample(frac=0.3, random_state=101)
                X_train_cv = train.drop(['TARGET','fulldate'], axis=1)
                y_train_cv = train['TARGET'].astype(int)
                X_train_tune = train_tune.drop(['TARGET','fulldate'], axis=1)
                y_train_tune = train_tune['TARGET'].astype(int)                
                
                X_train_cv, y_train_cv = resampling_tecnique(X_train_cv, y_train_cv)
                X_train_tune, y_train_tune = random_undersampling(X_train_tune, y_train_tune)

                print('X_train_cv shape: ', X_train_cv.shape)
                print('X_train_tune shape: ', X_train_tune.shape)
            else:
               
                print(f'Treino feito com os meses: {ts} até {ts+2}') # 11
            
                train = df_ML[df_ML['fulldate'].isin([ts, ts+1, ts+2])]  # ts+3, ts+4, ts+5, ts+6, ts+7, ts+8, ts+9, ts+10, ts+11
                train_tune = train.groupby('fulldate').sample(frac=0.3, random_state=101)
                X_train_cv = train.drop(['TARGET','fulldate'], axis=1)
                y_train_cv = train['TARGET'].astype(int)
                X_train_tune = train_tune.drop(['TARGET','fulldate'], axis=1)
                y_train_tune = train_tune['TARGET'].astype(int) 

                X_train_cv, y_train_cv = resampling_tecnique(X_train_cv, y_train_cv)
                X_train_tune, y_train_tune = random_undersampling(X_train_tune, y_train_tune)
                
                print('X_train_cv shape: ', X_train_cv.shape)
                print('X_train_tune shape: ', X_train_tune.shape)
            
            print(f'Testado com mês: {ts+3}')
            test = df_ML[df_ML['fulldate'] == ts+3] # 12
            # print(f'Teste Fulldate: {test["fulldate"].unique()}')
            # print(f'Ts: {ts+3}')
            X_test = test.drop(['TARGET', 'fulldate'], axis=1)

            X_test_pandas = pd.DataFrame(X_test, columns=X_test.columns)

            df_total = df_total.append(X_test_pandas)
            

            # full_xtest_df = pd.concat(X_test)

            full_test_count += X_test.shape[0]


            y_test = test['TARGET'].astype(int)
            print(f'X_test shape do mês {ts +3}: ', X_test.shape)
            #print(f'Teste feito com o mês: {ts+3}')  # 12

            
            #print y_train_cv data type
            
            y_pred = model(X_train_cv, y_train_cv, X_train_tune, y_train_tune, X_test, y_test)
            # print(f'Treino feito com os meses: {ts} até {ts+2}') # 11
            # print(f'Teste feito com o mês: {ts+3}')  # 12
            # print('X_train_cv shape: ', X_train_cv.shape)
            # print('X_train_tune shape: ', X_train_tune.shape)
            # print(f'Y_pred shape: {y_pred.shape}')
            #append accuracy variable to the accuracy array
            accuracy_array = np.append(accuracy_array, accuracy)
            #append precision variable to the precision array
            precision_array = np.append(precision_array, precision)
            #append recall variable to the recall array
            recall_array = np.append(recall_array, recall)
            #append f1 variable to the f1 array
            f1_array = np.append(f1_array, f1)
            
            # Get the model exported in the last iteration of the model
            model.features = features_selected
            model.paramethers = param
            model_exported = model

            


            probability_array = np.append(probability_array, y_proba)
            print('\nEnd Iteration\n')
            #pint barrier or * to separate the iterations
            print('******************************************************************************************')
        else: 
            print(f'\nFinish the for loop\n')
            print(f'Accuracy Mean of all iterations : {np.mean(accuracy_array)}')
            print(f'Precision Mean of all iterations : {np.mean(precision_array)}')
            print(f'Recall Mean of all iterations: {np.mean(recall_array)}')
            print(f'F1 mean of all iterations: {np.mean(f1_array)}')
            return X_train_cv, y_train_cv,X_test, y_test , probability_array, df_total, y_pred, features_selected, param, model_exported

    #print(f'\n\nNumber of test samples: {full_test_count}')
    

In [ ]:
#save model exported using joblib
def save_model(model, model_name):
    joblib.dump(model, model_name)
    print(f'\nModel {model_name} saved successfully!')

In [ ]:
def random_oversampling(x_train,y_train):
    ros = RandomOverSampler(random_state=101)
    X_train, y_train = ros.fit_resample(x_train, y_train)
    
    return X_train, y_train

def roSMOTE(x_train,y_train):
    os = SMOTE(random_state=101)
    X_train, y_train = os.fit_resample(x_train, y_train)
    
    return X_train, y_train

def random_undersampling(x_train,y_train):
    rus = RandomUnderSampler(random_state=101)
    X_train, y_train = rus.fit_resample(x_train, y_train)
    
    return X_train, y_train

In [ ]:
X_train_cv, y_train_cv,X_test, y_test , y_proba, df_total, y_pred, features_selected, param, model = cv_12months(df, LR, random_undersampling)   # Taking too long... What about removing all subcategory columns with all values 0?

In [ ]:
features_selected

## Recommendation System - Top 20 Subcategories for each Customer in the month of November



### Normal recommendation with model ran above

If you didn't run the model, ignore this. Run the sections below with the saved models.

In [ ]:
#aggregate the results of y_proba in tuples of 2
y_proba = y_proba.reshape(int(y_proba.shape[0]/2), 2)

In [ ]:
df_total['PROBABILITIES'] = y_proba.tolist()

In [ ]:
def recommend_fixed():
    df_full = df.groupby('fulldate').sample(frac=0.2, random_state=101)
    df_nov = df_full[df_full['fulldate']=='2022-11']
    #reset index of df_200k_oct
    df_nov = df_nov.reset_index(drop=True)
    y_proba_last = y_proba[-y_pred.shape[0]:]
    y_proba_df = pd.DataFrame(y_proba_last, columns=['NO BUY', 'BUY'])
    y_pred_df = pd.DataFrame(y_pred, columns=['Target'])
    joined_prob_pred = y_pred_df.join(y_proba_df)
    df_oct_joined = df_nov.join(joined_prob_pred)
    df_oct_joined = df_oct_joined[['CUSTOMER_ACCOUNT_NR_MASK', 'SUBCAT_CD_EXT', 'SEG_LIFESTYLE_CD',  'NO BUY', 'BUY', 'Target']]
    #select only the rows where Target = 1
    df_oct_joined = df_oct_joined[df_oct_joined['Target'] == 1]
    #group by customer account nr mask and sort by buy
    recs_grouped = df_oct_joined.groupby('CUSTOMER_ACCOUNT_NR_MASK').apply(lambda x: x.sort_values(by='BUY', ascending=False))
    #rename CUSTOMER_ACCOUNT_NR_MASK to CUSTOMER
    recs_grouped = recs_grouped.rename(columns={'CUSTOMER_ACCOUNT_NR_MASK': 'CUSTOMER'})
    top_subcategories = recs_grouped.groupby('CUSTOMER_ACCOUNT_NR_MASK').apply(lambda x: x.nlargest(20, 'BUY'))
    

    return recs_grouped, top_subcategories

In [ ]:
recs_grouped, top_subcategories = recommend_fixed()

In [ ]:
recs_grouped

### Recommendations using .joblib file

Independentemente do mês escolhido, as recomendações vão ser geradas com base nas features da última iteração do modelo

In [86]:
def recommend_joblib(month, joblib_file):

   df_full_job = df.groupby('fulldate').sample(frac=1, random_state=101)
   df_nov_job = df_full_job[df_full_job['fulldate']==month]

   
   test_job = df_sample[df_sample['fulldate'] == month] 
   X_test_job = test_job.drop(['TARGET', 'fulldate'], axis=1)

   X_test_pandas_job = pd.DataFrame(X_test_job, columns=X_test_job.columns)
   model_job = joblib.load(joblib_file)
   X_test_deploy_job = X_test_pandas_job[['CUST_NUM_TRANSACTIONS_MONTH', 'CUST_NUM_TRANSACTIONS_QUARTER',
      'CUST_NUM_TRANSACTIONS_SEMESTER', 'CUST_NUM_TRANSACTIONS_YEAR',
      'CUST_TOTAL_QTY_BOUGHT_MONTH', 'CUST_TOTAL_QTY_BOUGHT_QUARTER',
      'CUST_TOTAL_QTY_BOUGHT_YEAR', 'CUST_NUM_UNIQUE_SUBCAT_SEMESTER',
      'CUST_NUM_UNIQUE_SUBCAT_YEAR',
      'CUST_AVG_DAYS_SINCE_PRIOR_TRANSACTION_MONTH',
      'CUST_AVG_DAYS_SINCE_PRIOR_TRANSACTION_QUARTER',
      'CUST_AVG_DAYS_SINCE_PRIOR_TRANSACTION_SEMESTER',
      'CUST_AVG_DAYS_SINCE_PRIOR_TRANSACTION_YEAR',
      'CUST_AVG_BASKET_SIZE_SEMESTER', 'SUBCAT_NUM_TRANSACTIONS_YEAR',
      'SUBCAT_TOTAL_QTY_BOUGHT_QUARTER', 'SUBCAT_NUM_UNIQUE_CUST_MONTH',
      'CUSTSUBCAT_NUM_TRANSACTIONS_SEMESTER',
      'CUSTSUBCAT_TOTAL_QTY_BOUGHT_SEMESTER',
      'CUSTSUBCAT_AVG_DAYS_SINCE_PRIOR_TRANSACTION_MONTH',
      'CUSTSUBCAT_AVG_DAYS_SINCE_PRIOR_TRANSACTION_QUARTER',
      'CUSTSUBCAT_AVG_DAYS_SINCE_PRIOR_TRANSACTION_SEMESTER',
      'CUSTSUBCAT_AVG_DAYS_SINCE_PRIOR_TRANSACTION_YEAR', 'GENDER_F', 'GENDER_M', 'FAMILY_MEMBERS_(0, 0)',
      'FAMILY_MEMBERS_(1, 2)', 'FAMILY_MEMBERS_(3, 8)', 'CAT_CD_EXT_101',
      'CAT_CD_EXT_102', 'CAT_CD_EXT_103', 'CAT_CD_EXT_202', 'CAT_CD_EXT_203',
      'CAT_CD_EXT_302', 'CAT_CD_EXT_303', 'CAT_CD_EXT_304', 'CAT_CD_EXT_502',
      'CAT_CD_EXT_503', 'CAT_CD_EXT_504', 'CAT_CD_EXT_601', 'CAT_CD_EXT_603',
      'CAT_CD_EXT_604', 'CAT_CD_EXT_702', 'CAT_CD_EXT_801', 'CAT_CD_EXT_804',
      'CAT_CD_EXT_902', 'CAT_CD_EXT_1001', 'CAT_CD_EXT_1002',
      'CAT_CD_EXT_1402', 'CAT_CD_EXT_1403', 'CAT_CD_EXT_1701',
      'CAT_CD_EXT_1703', 'YEAR_2021', 'YEAR_2022']]
   y_proba_job = model_job.predict_proba(X_test_deploy_job)
   y_pred_job = model_job.predict(X_test_deploy_job)
   X_test_pandas_job['PROBABILITIES'] = y_proba_job.tolist()

   #reset index of df_200k_oct
   df_nov_job = df_nov_job.reset_index(drop=True)
   y_proba_last = y_proba_job[-y_pred_job.shape[0]:]
   y_proba_df = pd.DataFrame(y_proba_last, columns=['NO BUY', 'BUY'])
   y_pred_df = pd.DataFrame(y_pred_job, columns=['Target'])
   joined_prob_pred = y_pred_df.join(y_proba_df)
   df_oct_joined = df_nov_job.join(joined_prob_pred)
   df_oct_joined = df_oct_joined[['CUSTOMER_ACCOUNT_NR_MASK', 'SUBCAT_CD_EXT', 'SEG_LIFESTYLE_CD',  'NO BUY', 'BUY', 'Target']]
   #select only the rows where Target = 1
   df_oct_joined = df_oct_joined[df_oct_joined['Target'] == 1]
   #group by customer account nr mask and sort by buy
   recs_grouped = df_oct_joined.groupby('CUSTOMER_ACCOUNT_NR_MASK').apply(lambda x: x.sort_values(by='BUY', ascending=False))
   #rename CUSTOMER_ACCOUNT_NR_MASK to CUSTOMER
   recs_grouped_job = recs_grouped.rename(columns={'CUSTOMER_ACCOUNT_NR_MASK': 'CUSTOMER'})


   top20_subcategories_job = recs_grouped_job.groupby('CUSTOMER_ACCOUNT_NR_MASK').apply(lambda x: x.nlargest(1000, 'BUY'))
   

   return df_oct_joined, y_pred_job, recs_grouped_job, top20_subcategories_job





In [87]:
df_oct_joined, y_pred_job, recs_grouped_job, top20_subcategories_job = recommend_joblib('2022-11', 'RF.joblib')

In [81]:
df_oct_joined.shape

(45411, 6)

In [88]:
#Creates the stats for each category in the full recomendations dataset

freq_customer1 = recs_grouped_job['CUSTOMER'].value_counts().idxmax()

recs_for_freqc1 = recs_grouped_job[recs_grouped_job['CUSTOMER'] == freq_customer1]

new_recs = recs_grouped_job.reset_index()

new_recs = new_recs[['SUBCAT_CD_EXT','Target']]

total_stats_rec = new_recs.groupby('SUBCAT_CD_EXT').size().reset_index(name='FREQ_RECS_TOTAL')

# Calculate the total count
total_count = total_stats_rec['FREQ_RECS_TOTAL'].sum()

# Add a percentage column
total_stats_rec['PERC_RECS_TOTAL'] = (total_stats_rec['FREQ_RECS_TOTAL'] / total_count) * 100




(141, 3)

In [89]:
freq_customer = recs_grouped_job['CUSTOMER'].value_counts().idxmax()

recs_for_freqc = recs_grouped_job[recs_grouped_job['CUSTOMER'] == freq_customer]

#show the most frequent subcategories for each unique SEG_LIFESTYLE_CD
freq_subcats = pd.DataFrame(top20_subcategories_job.groupby('SEG_LIFESTYLE_CD')['SUBCAT_CD_EXT'].value_counts())
pd.set_option('display.max_rows', None)
#rename SUBCAT_CD_EXT to FREQUENCY
freq_subcats = freq_subcats.rename(columns={'SUBCAT_CD_EXT': 'FREQUENCY'})
#create new column with percentage of each subcategory
freq_subcats['PERCENTAGE'] = freq_subcats.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))

freq_subcats = freq_subcats.rename(columns={'FREQUENCY': 'FREQ_RECS_BY_LS', 'PERCENTAGE': 'PERC_RECS_BY_LS'})

#reset index from teste
teste_reseted = freq_subcats.reset_index()

In [91]:
teste_reseted.head()

,SEG_LIFESTYLE_CD,SUBCAT_CD_EXT,FREQ_RECS_BY_LS,PERC_RECS_BY_LS
0,1,100204,401,1.84
1,1,10101,397,1.82
2,1,100201,388,1.78
3,1,60401,379,1.74
4,1,30401,377,1.73


In [93]:
#print unique SUBCAT_CD_EXT
teste_reseted['SUBCAT_CD_EXT'].shape

(506,)

In [71]:
#merge df_oct_joined with total_stats_rec
df_oct_joined2 = df_oct_joined.merge(total_stats_rec, how='left', on='SUBCAT_CD_EXT')

df_oct_joined2.head()


,CUSTOMER_ACCOUNT_NR_MASK,SUBCAT_CD_EXT,SEG_LIFESTYLE_CD,NO BUY,BUY,Target,FREQ_RECS_TOTAL,PERC_RECS_TOTAL
0,13025896,50203,1,0.31,0.69,1,667,1.47
1,2231323,140304,2,0.21,0.79,1,636,1.40
2,59891701,20301,1,0.10,0.90,1,589,1.30
3,79547988,60305,1,0.43,0.57,1,98,0.22
4,9493177,140304,1,0.04,0.96,1,636,1.40


### Calculation of frequencies for subcategories (recommendations and total dataset)

In [18]:
##########é para copiar la p baixo

df_sample2 = df.groupby('fulldate').sample(frac=1, random_state=101)

#drop customer id column because we want to generalize the model, instead of trying to predict for each customer
df_sample2 = df_sample2.drop(columns=['CUSTOMER_ACCOUNT_NR_MASK','REG_AVG_DAYS_SINCE_PRIOR_TRANSACTION_MONTH'])

#df_500k.value_counts('fulldate').sort_index()

# Function to replace outliers with average value of the column
def replace_outliers_with_average(df_non_minmax, column):
    # Calculate the lower and upper bounds for outliers using the boxplot rule
    q1 = df_non_minmax[column].quantile(0.25)
    q3 = df_non_minmax[column].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    
    # Replace outliers with the average value of the column
    avg_value = df_non_minmax[column].mean()
    df_non_minmax[column] = df_non_minmax[column].apply(lambda x: avg_value if x < lower_bound or x > upper_bound else x)
    
    return df_non_minmax

# Detect outliers and replace them with average for each column in 'columns_to_check'
numerical_columns = ['CUST_NUM_TRANSACTIONS_MONTH', 'CUST_NUM_TRANSACTIONS_QUARTER',
       'CUST_NUM_TRANSACTIONS_SEMESTER', 'CUST_NUM_TRANSACTIONS_YEAR',
       'CUST_TOTAL_QTY_BOUGHT_MONTH', 'CUST_TOTAL_QTY_BOUGHT_QUARTER',
       'CUST_TOTAL_QTY_BOUGHT_SEMESTER', 'CUST_TOTAL_QTY_BOUGHT_YEAR',
       'CUST_NUM_UNIQUE_SUBCAT_MONTH', 'CUST_NUM_UNIQUE_SUBCAT_QUARTER',
       'CUST_NUM_UNIQUE_SUBCAT_SEMESTER', 'CUST_NUM_UNIQUE_SUBCAT_YEAR',
       'CUST_AVG_DAYS_SINCE_PRIOR_TRANSACTION_MONTH',
       'CUST_AVG_DAYS_SINCE_PRIOR_TRANSACTION_QUARTER',
       'CUST_AVG_DAYS_SINCE_PRIOR_TRANSACTION_SEMESTER',
       'CUST_AVG_DAYS_SINCE_PRIOR_TRANSACTION_YEAR',
       'CUST_AVG_BASKET_SIZE_MONTH', 'CUST_AVG_BASKET_SIZE_QUARTER',
       'CUST_AVG_BASKET_SIZE_SEMESTER', 'CUST_AVG_BASKET_SIZE_YEAR',
       'SUBCAT_NUM_TRANSACTIONS_MONTH', 'SUBCAT_NUM_TRANSACTIONS_QUARTER',
       'SUBCAT_NUM_TRANSACTIONS_SEMESTER', 'SUBCAT_NUM_TRANSACTIONS_YEAR',
       'SUBCAT_TOTAL_QTY_BOUGHT_MONTH', 'SUBCAT_TOTAL_QTY_BOUGHT_QUARTER',
       'SUBCAT_TOTAL_QTY_BOUGHT_SEMESTER', 'SUBCAT_TOTAL_QTY_BOUGHT_YEAR',
       'SUBCAT_NUM_UNIQUE_CUST_MONTH', 'SUBCAT_NUM_UNIQUE_CUST_QUARTER',
       'SUBCAT_NUM_UNIQUE_CUST_SEMESTER', 'SUBCAT_NUM_UNIQUE_CUST_YEAR',
       'CUSTSUBCAT_NUM_TRANSACTIONS_MONTH',
       'CUSTSUBCAT_NUM_TRANSACTIONS_QUARTER',
       'CUSTSUBCAT_NUM_TRANSACTIONS_SEMESTER',
       'CUSTSUBCAT_NUM_TRANSACTIONS_YEAR', 'CUSTSUBCAT_TOTAL_QTY_BOUGHT_MONTH',
       'CUSTSUBCAT_TOTAL_QTY_BOUGHT_QUARTER',
       'CUSTSUBCAT_TOTAL_QTY_BOUGHT_SEMESTER',
       'CUSTSUBCAT_TOTAL_QTY_BOUGHT_YEAR',
       'CUSTSUBCAT_AVG_DAYS_SINCE_PRIOR_TRANSACTION_MONTH',
       'CUSTSUBCAT_AVG_DAYS_SINCE_PRIOR_TRANSACTION_QUARTER',
       'CUSTSUBCAT_AVG_DAYS_SINCE_PRIOR_TRANSACTION_SEMESTER',
       'CUSTSUBCAT_AVG_DAYS_SINCE_PRIOR_TRANSACTION_YEAR']

for column in numerical_columns:
    df_sample2 = replace_outliers_with_average(df_sample2, column)
    
df_sample2.shape


scaler = MinMaxScaler()
df_sample2[numerical_columns] = scaler.fit_transform(df_sample2[numerical_columns])

df_sample2 = df_sample2.drop(columns=['QUARTER','SEMESTER','MONTH'])

# One-hot encoding categorical variables
df_sample2 = pd.get_dummies(df_sample2, columns=['GENDER','FAMILY_MEMBERS','YEAR'])


In [19]:
#select only rows where TARGET is equal to 1
df_only_1s = df_sample2[df_sample2['TARGET'] == 1]
#show most frequent SUBCAT_CD_EXT for each SEG_LIFESTYLE_CD
lifestyles_w_subcat = pd.DataFrame(df_only_1s.groupby('SEG_LIFESTYLE_CD')['SUBCAT_CD_EXT'].value_counts())

#instead of using the absolute number, use the percentage of each SUBCAT_CD_EXT for each SEG_LIFESTYLE_CD
lifestyles_w_subcat['PERC_TOTAL'] = lifestyles_w_subcat.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))

#rename the column SUBCAT_CD_EXT to FREQ_TOTAL
lifestyles_w_subcat.rename(columns={'SUBCAT_CD_EXT': 'FREQ_TOTAL'}, inplace=True)

#make a copy of lifestyles_w_subcat
lifestyles_w_subcat_copy = lifestyles_w_subcat.copy()

#reset the index
lifestyles_w_subcat.reset_index(inplace=True)



In [20]:
#Creates the stats for each category in the full recomendations dataset

freq_customer1 = recs_grouped_job['CUSTOMER'].value_counts().idxmax()

recs_for_freqc1 = recs_grouped_job[recs_grouped_job['CUSTOMER'] == freq_customer1]

new_recs = recs_grouped_job.reset_index()

new_recs = new_recs[['SUBCAT_CD_EXT','Target']]

total_stats_rec = new_recs.groupby('SUBCAT_CD_EXT').size().reset_index(name='FREQ_RECS_TOTAL')

# Calculate the total count
total_count = total_stats_rec['FREQ_RECS_TOTAL'].sum()

# Add a percentage column
total_stats_rec['PERC_RECS_TOTAL'] = (total_stats_rec['FREQ_RECS_TOTAL'] / total_count) * 100


total_stats_rec


#

,SUBCAT_CD_EXT,FREQ_RECS_TOTAL,PERC_RECS_TOTAL
0,10101,852,1.88
1,10102,777,1.71
2,10201,704,1.55
3,10202,430,0.95
4,10203,305,0.67
...,...,...,...
136,170305,5,0.01
137,170307,45,0.10
138,170309,11,0.02
139,170310,20,0.04


In [21]:
freq_customer = recs_grouped_job['CUSTOMER'].value_counts().idxmax()

recs_for_freqc = recs_grouped_job[recs_grouped_job['CUSTOMER'] == freq_customer]

#show the most frequent subcategories for each unique SEG_LIFESTYLE_CD
freq_subcats = pd.DataFrame(top20_subcategories_job.groupby('SEG_LIFESTYLE_CD')['SUBCAT_CD_EXT'].value_counts())
pd.set_option('display.max_rows', None)
#rename SUBCAT_CD_EXT to FREQUENCY
freq_subcats = freq_subcats.rename(columns={'SUBCAT_CD_EXT': 'FREQUENCY'})
#create new column with percentage of each subcategory
freq_subcats['PERCENTAGE'] = freq_subcats.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))

#reset index from teste
teste_reseted = freq_subcats.reset_index()

#join teste_reseted with recs_for_freqc on SEG_LIFESTYLE_CD and SUBCAT_CD_EXT
joined_both = teste_reseted.merge(recs_grouped_job, on=['SEG_LIFESTYLE_CD', 'SUBCAT_CD_EXT'])

joined_both = joined_both.merge(total_stats_rec, on=['SUBCAT_CD_EXT'])


#rename frequency to freq_recs and percentage to perc_recs
joined_both = joined_both.rename(columns={'FREQUENCY': 'FREQ_RECS_BY_LS', 'PERCENTAGE': 'PERC_RECS_BY_LS'})

#join joined_both with lifestyles_w_subcat on SEG_LIFESTYLE_CD and SUBCAT_CD_EXT
joined_wo_dsc = joined_both.merge(lifestyles_w_subcat, on=['SEG_LIFESTYLE_CD', 'SUBCAT_CD_EXT'])

ordenar_colunas = ['CUSTOMER', 'SEG_LIFESTYLE_CD', 'SUBCAT_CD_EXT', 'FREQ_RECS_BY_LS','FREQ_RECS_TOTAL', 'FREQ_TOTAL', 'PERC_RECS_BY_LS', 'PERC_RECS_TOTAL', 'PERC_TOTAL'] 

joined_wo_dsc = joined_wo_dsc[ordenar_colunas]

#order by perc_total
joined_wo_dsc = joined_wo_dsc.sort_values(by=['PERC_TOTAL'], ascending=False)

#join fulljoin with pdct_df on SUBCAT_CD_EXT
joined_wo_dsc = joined_wo_dsc.merge(pdct_df, on='SUBCAT_CD_EXT')

#drop duplicate rows
joined_wo_dsc = joined_wo_dsc.drop_duplicates()



In [22]:
#order joined_wo_dsc by CUSTOMER
joined_wo_dsc = joined_wo_dsc.sort_values(by=['CUSTOMER'], ascending=True)
joined_wo_dsc.head(10)


,CUSTOMER,SEG_LIFESTYLE_CD,SUBCAT_CD_EXT,FREQ_RECS_BY_LS,FREQ_RECS_TOTAL,FREQ_TOTAL,PERC_RECS_BY_LS,PERC_RECS_TOTAL,PERC_TOTAL,SUBCAT_DSC_EXT
4852140,37912,2,60406,53,589,1595,0.87,1.30,1.03,060406 - sacos lixo & luvas
8878254,37912,2,30208,29,170,613,0.48,0.37,0.39,030208 - energéticas
6588024,37912,2,80406,42,370,1104,0.69,0.81,0.71,080406 - magros
8748257,37912,2,20205,9,241,742,0.15,0.53,0.48,020205 - sazonais
9108219,37912,2,60107,2,86,272,0.03,0.19,0.18,060107 - intensificad perfume
6424292,37912,2,80405,75,361,1684,1.23,0.79,1.08,080405 - infantis
8938782,37912,2,50403,3,153,639,0.05,0.34,0.41,050403 - perfumaria bebé
3637997,37912,2,50203,50,667,2064,0.82,1.47,1.33,050203 - saúde e bem-estar
1157010,37912,2,10303,154,778,2638,2.53,1.71,1.70,010303 - vegetais em conserva
86242,37912,2,10102,179,777,2820,2.94,1.71,1.81,010102 - snacks salgados


### Compare entire dataframe with recommendations by Lifestage Segment

In [75]:
def lifestage_comparison():
    #from def_oct_joined group by seg_lifestyle_cd and sort by buy
    df_nov_joined = df_oct_joined.groupby('SEG_LIFESTYLE_CD').apply(lambda x: x.sort_values(by='BUY', ascending=False))

    #rename SEG_LIFESTYLE_CD to LYFESTYLE_SEGMENT
    df_nov_joined = df_nov_joined.rename(columns={'SEG_LIFESTYLE_CD': 'LYFESTYLE_SEGMENT'})

    #select 20 rows from every group
    topmais = df_nov_joined.groupby('SEG_LIFESTYLE_CD').apply(lambda x: x.nlargest(1000, 'BUY'))
    #show the most frequent subcategories for each unique SEG_LIFESTYLE_CD
    freq_subcats2 = pd.DataFrame(top20_subcategories_job.groupby('SEG_LIFESTYLE_CD')['SUBCAT_CD_EXT'].value_counts())
    pd.set_option('display.max_rows', None)
    #rename SUBCAT_CD_EXT to FREQUENCY
    freq_subcats2 = freq_subcats2.rename(columns={'SUBCAT_CD_EXT': 'FREQUENCY'})
    #create new column with percentage of each subcategory
    freq_subcats2['PERCENTAGE'] = freq_subcats2.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))

    topmais_drop = topmais.reset_index(level=1, drop=True)

    #show the most frequent subcategories for each unique SEG_LIFESTYLE_CD
    freq_subcats2 = pd.DataFrame(top20_subcategories_job.groupby('SEG_LIFESTYLE_CD')['SUBCAT_CD_EXT'].value_counts())
    pd.set_option('display.max_rows', None)
    #rename SUBCAT_CD_EXT to FREQUENCY
    freq_subcats2 = freq_subcats2.rename(columns={'SUBCAT_CD_EXT': 'FREQUENCY'})
    #create new column with percentage of each subcategory
    freq_subcats2['PERCENTAGE'] = freq_subcats2.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))

    #reset index from teste
    teste_reseted2 = freq_subcats2.reset_index()

    #join teste_reseted with recs_for_freqc on SEG_LIFESTYLE_CD and SUBCAT_CD_EXT
    joined_both2 = teste_reseted2.merge(topmais_drop, on=['SEG_LIFESTYLE_CD', 'SUBCAT_CD_EXT'])

    #rename frequency to freq_recs and percentage to perc_recs
    joined_both2 = joined_both2.rename(columns={'FREQUENCY': 'FREQ_RECS_BY_LS', 'PERCENTAGE': 'PERC_RECS_BY_LS'})

    #join joined_both with lifestyles_w_subcat on SEG_LIFESTYLE_CD and SUBCAT_CD_EXT
    joined_wo_dsc2 = joined_both2.merge(lifestyles_w_subcat, on=['SEG_LIFESTYLE_CD', 'SUBCAT_CD_EXT'])

    joined_wo_dsc2 = joined_wo_dsc2.merge(pdct_df, on='SUBCAT_CD_EXT')

    joined_wo_dsc2 = joined_wo_dsc2.merge(total_stats_rec, on=['SUBCAT_CD_EXT'])

    ordenar_colunas = ['SEG_LIFESTYLE_CD', 'SUBCAT_CD_EXT', 'SUBCAT_DSC_EXT', 'FREQ_RECS_BY_LS', 'FREQ_RECS_TOTAL', 'FREQ_TOTAL', 'PERC_RECS_BY_LS', 'PERC_RECS_TOTAL', 'PERC_TOTAL'] 

    joined_wo_dsc2 = joined_wo_dsc2[ordenar_colunas]

    #group by seg_lifestyle_cd
    joined_wo_dsc2 = joined_wo_dsc2.groupby('SEG_LIFESTYLE_CD').apply(lambda x: x.sort_values(by='PERC_TOTAL', ascending=True))

    #rename column name SEG_LIFESTYLE_CD to LIFESTYLE_SEGMENT
    joined_wo_dsc2 = joined_wo_dsc2.rename(columns={'SEG_LIFESTYLE_CD': 'LIFESTYLE_SEGMENT'})

    #order by perc_total
    # joined_wo_dsc2 = joined_wo_dsc2.sort_values(by=['PERC_TOTAL'], ascending=True)

    # #order index SEG_LIFESTYLE_CD descending
    # joined_wo_dsc2 = joined_wo_dsc2.sort_index(ascending=True)
    
    

    #drop duplicates from joined_wo_dsc2
    joined_wo_dsc2 = joined_wo_dsc2.drop_duplicates(subset=['LIFESTYLE_SEGMENT', 'SUBCAT_CD_EXT'], keep='first')

    #drop column LIFESTYLE_SEGMENT
    joined_wo_dsc2 = joined_wo_dsc2.drop(columns=['LIFESTYLE_SEGMENT'])

    #order jojned_wo_dsc2
    #joined_wo_dsc2 = joined_wo_dsc2.sort_values(by=['PERC_TOTAL'], ascending=False)

    

    return joined_wo_dsc2



In [73]:
ls_comparison2 = lifestage_comparison()

In [25]:
#select first 5 rows from each index
ls_comparison2.groupby('SEG_LIFESTYLE_CD').head(3)

SUBCAT_CD_EXT                 SUBCAT_DSC_EXT  \
SEG_LIFESTYLE_CD                                                        
1                661506          80405              080405 - infantis   
                 638480          50401               050401 - fraldas   
                 654603          30203           030203 - Ref com gás   
2                676346          80407               080407 - bifidus   
                 547990          80409         080409 - Iog Proteicos   
                 675487         140205      140205 - farinhas p/ bebé   
3                683842         140202     140202 - outros alim. bebé   
                 684038          80101    080101 - leite pasteurizado   
                 683917          80414           080414 - Sem Lactose   
4                632849          20203  020203 - drops, gomas e rebuç   
                 510145          30202               030202 - ice tea   
                 683739         140206                 140206 - papas   

                         FREQ_RECS_BY_LS  FREQ_RECS_TOTAL  FREQ_TOTAL  \
SEG_LIFESTYLE_CD                                                        
1                661506               39              361        1042   
                 638480               58              283        1120   
                 654603               47              402        1241   
2                676346               19              212         696   
                 547990               46              380         938   
                 675487               25              205         976   
3                683842                4               85         106   
                 684038                2               47         126   
                 683917                3              112         140   
4                632849                1              502           4   
                 510145                1              544           4   
                 683739                1              117           4   

                         PERC_RECS_BY_LS  PERC_RECS_TOTAL  PERC_TOTAL  
SEG_LIFESTYLE_CD                                                       
1                661506             0.44             0.79        0.48  
                 638480             0.66             0.62        0.51  
                 654603             0.54             0.89        0.57  
2                676346             0.31             0.47        0.45  
                 547990             0.76             0.84        0.60  
                 675487             0.41             0.45        0.63  
3                683842             0.11             0.19        0.11  
                 684038             0.05             0.10        0.14  
                 683917             0.08             0.25        0.15  
4                632849             2.50             1.11        0.43  
                 510145             2.50             1.20        0.43  
                 683739             2.50             0.26        0.43

### Final Recommendations for each Customer

In [135]:
df_oct_joined, y_pred_job, recs_grouped_job, top20_subcategories_job_deploy = recommend_joblib('2022-11', 'RF.joblib')

In [140]:
def compare_customer_recommendations():
    # freq_customer = recs_grouped_job['CUSTOMER'].value_counts().idxmax()

    # recs_for_freqc = recs_grouped_job[recs_grouped_job['CUSTOMER'] == freq_customer]

    #show the most frequent subcategories for each unique SEG_LIFESTYLE_CD
    freq_subcats = pd.DataFrame(top20_subcategories_job_deploy.groupby('SEG_LIFESTYLE_CD')['SUBCAT_CD_EXT'].value_counts())
    pd.set_option('display.max_rows', None)
    #rename SUBCAT_CD_EXT to FREQUENCY
    freq_subcats = freq_subcats.rename(columns={'SUBCAT_CD_EXT': 'FREQUENCY'})
    #create new column with percentage of each subcategory
    freq_subcats['PERCENTAGE'] = freq_subcats.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))

    #reset index from teste
    teste_reseted = freq_subcats.reset_index()

    #join teste_reseted with recs_for_freqc on SEG_LIFESTYLE_CD and SUBCAT_CD_EXT
    joined_both = teste_reseted.merge(recs_grouped_job, on=['SEG_LIFESTYLE_CD', 'SUBCAT_CD_EXT'])

    #rename frequency to freq_recs and percentage to perc_recs
    joined_both = joined_both.rename(columns={'FREQUENCY': 'FREQ_RECS_BY_LS', 'PERCENTAGE': 'PERC_RECS_BY_LS'})

    joined_both = joined_both.merge(pdct_df, on='SUBCAT_CD_EXT')


    #join joined_both with lifestyles_w_subcat on SEG_LIFESTYLE_CD and SUBCAT_CD_EXT
    joined_wo_dsc = joined_both.merge(lifestyles_w_subcat, on=['SEG_LIFESTYLE_CD', 'SUBCAT_CD_EXT'])

    joined_wo_dsc = joined_wo_dsc.merge(total_stats_rec, on=['SUBCAT_CD_EXT'])

    ordenar_colunas = ['CUSTOMER', 'SEG_LIFESTYLE_CD', 'SUBCAT_CD_EXT','SUBCAT_DSC_EXT', 'FREQ_RECS_BY_LS', 'FREQ_RECS_TOTAL' , 'FREQ_TOTAL', 'PERC_RECS_BY_LS', 'PERC_RECS_TOTAL', 'PERC_TOTAL'] 

    customer = joined_wo_dsc[ordenar_colunas]

    

    #in the customer df, creates a column with the difference between perc_total and perc_recs
    customer['DIFFERENCE_PERC_BY_LS'] = customer['PERC_RECS_BY_LS'] - customer['PERC_TOTAL']
    customer['DIFFERENCE_PERC_TOTAL'] = customer['PERC_RECS_TOTAL'] - customer['PERC_TOTAL']


    #order customer by perc_total
    customer = customer.sort_values(by=['DIFFERENCE_PERC_TOTAL'], ascending=False)

    #drop duplicates from customer
    customer = customer.drop_duplicates(subset=['CUSTOMER', 'SEG_LIFESTYLE_CD', 'SUBCAT_CD_EXT'], keep='first')

    # #order DIFFERENCE_PERC_TOTAL descending
    final_recommendation = customer.groupby('CUSTOMER').apply(lambda x: x.sort_values('DIFFERENCE_PERC_TOTAL', ascending=False).head(20))

    #rename CUSTOMER to CUSTOMER_ID
    final_recommendation = final_recommendation.rename(columns={'CUSTOMER': 'CUSTOMER_ID'})

    return customer, final_recommendation


In [141]:
customer_rec, final_recommendation_rec = compare_customer_recommendations()

In [143]:
#select first 3 rows from each index
final_recommendation_rec.groupby('CUSTOMER_ID').head(1)

CUSTOMER_ID  SEG_LIFESTYLE_CD  SUBCAT_CD_EXT  \
CUSTOMER                                                          
37912     4714753        37912                 2          60406   
          6950966        37912                 2          80409   
          3398238        37912                 2          50303   
177299    5675841       177299                 1          20301   
          7114587       177299                 1          10207   
          5542338       177299                 1          60307   
179359    5227035       179359                 2          50301   
          6416891       179359                 2          50302   
          6951086       179359                 2          80409   
202298    5585586       202298                 2          60307   
          4714861       202298                 2          60406   
          5870599       202298                 2         100205   
295501    5585607       295501                 2          60307   
          2824054       295501                 2          10204   
          7128544       295501                 2          10207   
396634    5585902       396634                 2          60307   
          6417411       396634                 2          50302   
          7412487       396634                 2          60306   
423754    1158127       423754                 1          20204   
          6200948       423754                 1          20305   
          7144054       423754                 1          20203   
457300    5610275       457300                 3          60307   
          7538886       457300                 3          60405   
          8369271       457300                 3          60402   
471864    2496517       471864                 3          20306   
          5255027       471864                 3          50301   
          5610318       471864                 3          60307   
608850    1158439       608850                 1          20204   
          6201143       608850                 1          20305   
          7144382       608850                 1          20203   
681704    2824106       681704                 2          10204   
          5227373       681704                 2          50301   
          3399555       681704                 2          50303   
682808    2497068       682808                 3          20306   
          5255276       682808                 3          50301   
          4652549       682808                 3          60302   
731884    2497163       731884                 3          20306   
          5610592       731884                 3          60307   
          4652860       731884                 3          60302   
947350    6417413       947350                 2          50302   
          2612476       947350                 2          10201   
          7413303       947350                 2          60306   
1051535   1158485      1051535                 1          20204   
          7144430      1051535                 1          20203   
          7635878      1051535                 1         100101   
1072151   7012812      1072151                 2          10202   
          4606606      1072151                 2          60302   
          4715167      1072151                 2          60406   
1119779   6951315      1119779                 2          80409   
          3400130      1119779                 2          50303   
          4972574      1119779                 2          50202   
1133209   6923201      1133209                 3          30403   
          4430656      1133209                 3          90201   
          7539191      1133209                 3          60405   
1172015   1158664      1172015                 1          20204   
          6201167      1172015                 1          20305   
          7635893      1172015                 1         100101   
1242153   7128573      1242153                 2          10207   

## Deployment

December 2022 does not have the TARGET value. Using the best model/setup used for training, the TARGET value is predicted


In [26]:
X_test_deploy = df_sample[df_sample['fulldate'] == '2022-12']
X_test_deploy = X_test_deploy.drop(columns=['TARGET', 'fulldate'])
#from df_sample print number of rows with fulldate = '2022-12' 


In [27]:
df_oct_joined, y_pred_job, recs_grouped_job, top20_subcategories_job_deploy = recommend_joblib('2022-12', 'RF.joblib')

In [28]:
y_pred_job.shape[0] == X_test_deploy.shape[0]

True

In [50]:
def compare_customer():
    freq_customer = recs_grouped_job['CUSTOMER'].value_counts().idxmax()

    recs_for_freqc = recs_grouped_job[recs_grouped_job['CUSTOMER'] == freq_customer]

    #show the most frequent subcategories for each unique SEG_LIFESTYLE_CD
    freq_subcats = pd.DataFrame(top20_subcategories_job_deploy.groupby('SEG_LIFESTYLE_CD')['SUBCAT_CD_EXT'].value_counts())
    pd.set_option('display.max_rows', None)
    #rename SUBCAT_CD_EXT to FREQUENCY
    freq_subcats = freq_subcats.rename(columns={'SUBCAT_CD_EXT': 'FREQUENCY'})
    #create new column with percentage of each subcategory
    freq_subcats['PERCENTAGE'] = freq_subcats.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))

    #reset index from teste
    teste_reseted = freq_subcats.reset_index()

    #join teste_reseted with recs_for_freqc on SEG_LIFESTYLE_CD and SUBCAT_CD_EXT
    joined_both = teste_reseted.merge(recs_for_freqc, on=['SEG_LIFESTYLE_CD', 'SUBCAT_CD_EXT'])

    #rename frequency to freq_recs and percentage to perc_recs
    joined_both = joined_both.rename(columns={'FREQUENCY': 'FREQ_RECS_BY_LS', 'PERCENTAGE': 'PERC_RECS_BY_LS'})

    joined_both = joined_both.merge(pdct_df, on='SUBCAT_CD_EXT')


    #join joined_both with lifestyles_w_subcat on SEG_LIFESTYLE_CD and SUBCAT_CD_EXT
    joined_wo_dsc = joined_both.merge(lifestyles_w_subcat, on=['SEG_LIFESTYLE_CD', 'SUBCAT_CD_EXT'])

    joined_wo_dsc = joined_wo_dsc.merge(total_stats_rec, on=['SUBCAT_CD_EXT'])

    ordenar_colunas = ['CUSTOMER', 'SEG_LIFESTYLE_CD', 'SUBCAT_CD_EXT','SUBCAT_DSC_EXT', 'FREQ_RECS_BY_LS', 'FREQ_RECS_TOTAL' , 'FREQ_TOTAL', 'PERC_RECS_BY_LS', 'PERC_RECS_TOTAL', 'PERC_TOTAL'] 

    customer = joined_wo_dsc[ordenar_colunas]

    

    #in the customer df, creates a column with the difference between perc_total and perc_recs
    customer['DIFFERENCE_PERC_BY_LS'] = customer['PERC_RECS_BY_LS'] - customer['PERC_TOTAL']
    customer['DIFFERENCE_PERC_TOTAL'] = customer['PERC_RECS_TOTAL'] - customer['PERC_TOTAL']


    #order customer by perc_total
    customer = customer.sort_values(by=['DIFFERENCE_PERC_TOTAL'], ascending=False)

    #drop duplicates from customer
    customer = customer.drop_duplicates(subset=['CUSTOMER', 'SEG_LIFESTYLE_CD', 'SUBCAT_CD_EXT'], keep='first')

    #order customer by perc_total
    customer_sorted_asc = customer.sort_values(by=['PERC_TOTAL'], ascending=True)

    #from customer_sorted_asc drop rows where DIFFERENCE_PERC_TOTAL is <= 0
    customer_sorted_asc = customer_sorted_asc[customer_sorted_asc['DIFFERENCE_PERC_TOTAL'] > 0.00]

    #order DIFFERENCE_PERC_TOTAL descending
    final_recommendation = customer_sorted_asc.sort_values(by=['DIFFERENCE_PERC_TOTAL'], ascending=False).head(20)


    return customer, final_recommendation


In [51]:
customer, final_recommendation = compare_customer()

In [52]:
final_recommendation

,CUSTOMER,SEG_LIFESTYLE_CD,SUBCAT_CD_EXT,SUBCAT_DSC_EXT,FREQ_RECS_BY_LS,FREQ_RECS_TOTAL,FREQ_TOTAL,PERC_RECS_BY_LS,PERC_RECS_TOTAL,PERC_TOTAL,DIFFERENCE_PERC_BY_LS,DIFFERENCE_PERC_TOTAL
3606,13031606,1,20204,020204 - doçaria impulso,182,784,2947,2.06,1.73,1.34,0.72,0.38
5434,13031606,1,20305,020305 - bolos,107,585,2106,1.21,1.29,0.96,0.25,0.33
9746,13031606,1,20203,"020203 - drops, gomas e rebuç",68,502,1710,0.77,1.11,0.78,-0.01,0.33
11721,13031606,1,80405,080405 - infantis,39,361,1042,0.44,0.79,0.48,-0.03,0.32
11132,13031606,1,100101,100101 - óleos,49,489,1764,0.55,1.08,0.80,-0.25,0.27
5207,13031606,1,20301,020301 - bol. tradicionais,107,589,2247,1.21,1.30,1.03,0.19,0.27
12765,13031606,1,10207,010207 - sal,22,449,1581,0.25,0.99,0.72,-0.47,0.27
5028,13031606,1,50402,050402 - toalhitas bebé&crian,114,579,2246,1.29,1.28,1.02,0.27,0.25
11976,13031606,1,60307,060307 - acessórios limpeza,37,505,1914,0.42,1.11,0.87,-0.45,0.24
4781,13031606,1,10302,010302 - conservas carne,129,620,2496,1.46,1.37,1.14,0.32,0.23


## Visualizations

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

x = np.arange(10)
y = np.random.randint(-10, 10, 10)

mask = y < 0

plt.bar(x[mask], y[mask], color='red')
plt.bar(x[~mask], y[~mask], color='green')

plt.title('Graph with negative and positive values')
plt.xlabel('x')
plt.ylabel('y')

plt.show()